In [34]:
!pip install nmslib

     |████████████████████████████████| 13.0MB 251kB/s 
     |████████████████████████████████| 296kB 45.7MB/s 


In [30]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u252-b09-1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.


In [0]:
# Let's import the libraries we will need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [0]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [0]:
data_path = '/content/drive/My Drive/Questions.csv'

In [10]:
df = pd.read_csv(data_path, encoding="ISO8859-1")
df = df[['Id', 'Title', 'Body']]

# df = df.assign(Text=lambda x: x.Title + x.Body)

df.head(5)

,Id,Title,Body
0,80,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [32]:
df.shape

(1264216, 3)

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
    ngram_range=(1, 2))
X_tfidf = tfidf.fit_transform(df['Title'])
X_tfidf

<1264216x151918 sparse matrix of type '<class 'numpy.float64'>'
	with 10038385 stored elements in Compressed Sparse Row format>

In [52]:
X_tfidf

<1264216x151918 sparse matrix of type '<class 'numpy.float64'>'
	with 10038385 stored elements in Compressed Sparse Row format>

In [55]:
import nmslib

index = nmslib.init(method='hnsw', space='cosinesimil_sparse', data_type=nmslib.DataType.SPARSE_VECTOR)
index.addDataPointBatch(X_tfidf)

1264216

In [0]:
index_time_params = {'M': 30, 'indexThreadQty': 8, 'efConstruction': 100, 'post' : 0}
index.createIndex(index_time_params)

In [0]:
nbrs = index.knnQueryBatch(X_tfidf, k = 10, num_threads = 8)

In [66]:
nbrs[1]

(array([     1, 932497,  47348, 122096, 237084,  31952,  25700,  23422,
        988351,  28247], dtype=int32),
 array([0.        , 0.4213655 , 0.5109766 , 0.5228994 , 0.559223  ,
        0.5650945 , 0.57016087, 0.5836657 , 0.59998786, 0.6076    ],
       dtype=float32))

In [0]:
import pickle
with open('neighbors-10.pickle', 'wb') as handle:
    pickle.dump(nbrs, handle)

In [75]:
os.path.getsize('neighbors-10.pickle')

245259873

In [0]:
# with open('neighbors-10.pickle', 'rb') as readin:
#     b = pickle.load(readin)

In [77]:
# b[0]

(array([      0,  224837, 1168911, 1092386,  237304,   51271,  645549,
        1077042,  595294,  973721], dtype=int32),
 array([0.        , 0.4645568 , 0.53756285, 0.5428689 , 0.5466547 ,
        0.56866705, 0.5771997 , 0.57787925, 0.5779282 , 0.5971027 ],
       dtype=float32))

In [0]:
# from sklearn.neighbors import LSHForest

# lshf = LSHForest(n_neighbors=10)
# lsdf.fit(X_tfidf)

In [0]:
# encodings = df.apply(lambda x: [x.Id, model.encode(x.Title)], axis=1, result_type='expand')

In [0]:
# feature_vec = model.encode(df['Title'].tolist(), show_progress_bar=True, convert_to_numpy=True)